# Bibliotecas

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



from matplotlib.pyplot import plot, figure
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import arrow
from datetime import datetime as dt
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
import scipy
import scipy.stats
import pylab 
import warnings
warnings.filterwarnings("ignore")

# Define o tamanho das figuras
matplotlib.rcParams['figure.figsize'] = (18.0, 8.0)

## Modelo para prever as vendas semanais para diferentes departamentos de diversas lojas de um grande varejista

## Load e Join dos dados

In [ ]:
features_data = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
store_data = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
test_data = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
train_data = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')

# Indicadora de base de teste
test_data['is_test'] = 1
train_data['is_test'] = 0

# Junta as bases
data = pd.DataFrame(train_data.append(test_data)).sort_values('Date')

# Junta as features
data = pd.merge(data, features_data.drop('IsHoliday', axis=1), how="left", on=['Store','Date'])
data = pd.merge(data, store_data, how="left", on=['Store'])

## Construção de novas features

In [ ]:
data['IsHoliday'] = [ '1' if value == True else '0' for row, value in enumerate( data.IsHoliday ) ]

# Converte o campo Date de string para tipo Data
data["Date2"] = [ dt.strptime( value, '%Y-%m-%d') for row, value in enumerate(data.Date) ] 

# Função para retornar a semana do mes
def week_number_of_month(date_value):
    if( dt.strftime( date_value , '%b' ) == "Jan" ):
        out = date_value.isocalendar()[1]
    else:
        out = date_value.isocalendar()[1] - date_value.replace(day=1).isocalendar()[1] + 1
    return (out)

# Variáveis adicionais
data["week_month"] = [ week_number_of_month(value) for row, value in enumerate(data["Date2"])   ] 
data["month"] = [ dt.strftime( value , '%m-%b' ) for row, value in enumerate(data.Date2) ]

data2 = data

### Ajustes para vendas negativas

Olhando para o resumo descritivo da tabela abaixo, podemos ver que, para algumas observações, os valores das vendas são negativos. Pela natureza do dado, assumimos que esse valor não pode ser zero. Faremos a sua substituição pela média de vendas para o mesmo departamento e semana do mês.

In [ ]:
print(data.describe())

In [ ]:
# Backup antes do join
data = data2

# Cálculo da média por loja, departamento e semana do mês
df_medias_dept_week = data.groupby([ 'Store', 'Dept', 'week_month' ]).mean(['Weekly_Sales'])['Weekly_Sales'].reset_index().rename( columns={'Weekly_Sales': 'Weekly_Sales_Mean_week'} )

# Cálculo da média por loja e departamento
df_medias_dept = data.groupby([ 'Store', 'Dept']).mean(['Weekly_Sales'])['Weekly_Sales'].reset_index().rename( columns={'Weekly_Sales': 'Weekly_Sales_Mean'} )

# Salva uma cópia das vendas originais
data['Weekly_Sales_Original'] = data['Weekly_Sales']

# Join da base de vendas com as médias
data = data.merge( df_medias_dept_week, how='left', on = ['Store', 'Dept', 'week_month'],  )\
           .merge( df_medias_dept, how='left', on = ['Store', 'Dept'] )

# Substituição das vendas zeradas ou negativas pela média 
# Pode ser que o departamento nao tenha informacao para aquela semana, entao substitui pela media do departamento da loja
data.Weekly_Sales = [ data.Weekly_Sales_Mean_week[row] if value < 0 else value for row, value in enumerate(data.Weekly_Sales) ]
data.Weekly_Sales = [ data.Weekly_Sales_Mean[row] if value < 0 else value for row, value in enumerate(data.Weekly_Sales) ]


# 194 valores de Weekly_Sales ainda permanecem negativos, entao excluí da base
data.Weekly_Sales = [ np.nan if value <= 0 else value for row, value in enumerate(data.Weekly_Sales) ]
data = data[ (( ~(np.isnan(data.Weekly_Sales))) & (data.is_test == 0)) | (data.is_test == 1) ]

# Deleta as colunas de médias utilizadas para a substituição
data = data.drop(['Weekly_Sales_Mean', 'Weekly_Sales_Mean_week'], axis = 1)

# Redefine as bases de teste e treino
test_data = data[ data.is_test == 1 ]
train_data = data[ data.is_test == 0 ]

Note que, em geral, as substituições afetam somente alguns dias, lojas e departamentos específicos, essa diferença não é significativa olhando para o macro (gráfico abaixo).

In [ ]:
df_soma = train_data.groupby('Date').sum()

plt.plot( df_soma.Weekly_Sales )
plt.title("Vendas Ajustada vs Vendas Original")
plt.plot( df_soma.Weekly_Sales_Original )
plt.legend(["Vendas Ajustadas", "Vendas Originais"])

### Dados faltantes

Apliquei o mesmo processo de substituição dos dados faltantes feito acima para as covariáveis MarkDown1 a MarkDown5, pois essas variáveis apresentaram uma alta taxa de missings (tabela abaixo) e, por isso, o processo de substituição é mais adequado; outra alternativa seria a remoção delas do modelo. Além disso, retirei as observações nulas de CPI e Unemployment, pois representam um percentual baixo.

In [ ]:
### Verifição dos nulos
colunas = data.columns.to_list()
print(
    pd.DataFrame(
        { 'nulos' : [ 1-( len(data[i].dropna())/len(data)) for i in  colunas], 
          'coluna' : colunas 
        }
    )
)

In [ ]:
# Processo de substituição dos nulos
cols_replace = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

df_medias_store_MarkDown = train_data.groupby('Store').mean()[cols_replace].reset_index()
df_medias_geral_MarkDown = train_data.mean()[cols_replace]


data = data.merge( df_medias_store_MarkDown, how='left', on = ['Store'], suffixes=('', '_Store_Mean')  )

# Substitui dados nulos pela média da loja
data.MarkDown1 = [ data.MarkDown1_Store_Mean[row] if np.isnan(value) else value for row, value in enumerate(data.MarkDown1) ]
data.MarkDown2 = [ data.MarkDown2_Store_Mean[row] if np.isnan(value) else value for row, value in enumerate(data.MarkDown2) ]
data.MarkDown3 = [ data.MarkDown3_Store_Mean[row] if np.isnan(value) else value for row, value in enumerate(data.MarkDown3) ]
data.MarkDown4 = [ data.MarkDown4_Store_Mean[row] if np.isnan(value) else value for row, value in enumerate(data.MarkDown4) ]
data.MarkDown5 = [ data.MarkDown5_Store_Mean[row] if np.isnan(value) else value for row, value in enumerate(data.MarkDown5) ]

# Se ainda tiver nulos, substitui pela média geral
data.MarkDown1 = [ df_medias_geral_MarkDown[0] if np.isnan(value) else value for row, value in enumerate(data.MarkDown1) ]
data.MarkDown2 = [ df_medias_geral_MarkDown[1] if np.isnan(value) else value for row, value in enumerate(data.MarkDown2) ]
data.MarkDown3 = [ df_medias_geral_MarkDown[2] if np.isnan(value) else value for row, value in enumerate(data.MarkDown3) ]
data.MarkDown4 = [ df_medias_geral_MarkDown[3] if np.isnan(value) else value for row, value in enumerate(data.MarkDown4) ]
data.MarkDown5 = [ df_medias_geral_MarkDown[4] if np.isnan(value) else value for row, value in enumerate(data.MarkDown5) ]

# Deleta as médias utilizadas para a substituição
data = data.drop(['MarkDown1_Store_Mean', 'MarkDown2_Store_Mean', 'MarkDown3_Store_Mean', 'MarkDown4_Store_Mean', 'MarkDown5_Store_Mean'], 1)

# Deleta os nulos das colunas CPI e Unemployment
data = data.dropna( subset=['CPI', 'Unemployment'], how = 'any' )

# Redefine as bases de teste e treino
test_data = data[ data.is_test == 1 ]
train_data = data[ data.is_test == 0 ]

## Escolha da Técnica de Modelagem

Há pelo menos duas abordagens de modelagem que pode ser seguido com o trabalho:
  - **Séries Temporais:** Modelagem da variável "Venda" como uma variável aleatória dependente do tempo, identificando o seu comportamento ao longo do tempo e ajustando um modelo de previsão. Um modelo clássico seria o ARIMA ou o SARIMA (considera períodos sazonais)
  
  - **Modelos de regressão:** Uma abordagem via regressão incorpora dados explicativos à variável resposta (vendas) através de uma função de ligação. Através da identificação da distribuição da variável resposta, podemos aplicar um MLG (modelo linear geral) para explicar a venda futura.
  
  
Como possuímos algumas variáveis explicativas, como indicador de feriado, identificação da loja e departamento da venda, por simplicidade e menor tempo de entrega, optei por seguir com a modelagem via regressão.

No gráfico de linhas da série das vendas semanais, conseguimos ver períodos com maiores e menores valores absolutos de venda. 

A influência de feriados, dos meses de novembro e dezembro e da semana do mês, nas vendas, ficam evidentes nos gráficos abaixo.

In [ ]:
# Gráfico da Série de Vendas
weekly_sales = train_data.groupby(["Date","IsHoliday"]).sum().reset_index().sort_values("Date")
#weekly_sales["is_holiday"] = ["Sim" if value > 0 else "Nao" for row, value in enumerate( weekly_sales['IsHoliday'].astype(int) )]

# Tema do gráfico 
sns.set_theme(style='darkgrid')

# Divide a figura em 2 gráficos
plt.figure(1)

#
plt.subplot(211)
sns.lineplot(data = train_data, y="Weekly_Sales", x = "Date").set(title = "Vendas Totais")

plt.subplot(212)
sns.lineplot(data = weekly_sales, y="Weekly_Sales", x = "Date", hue = "IsHoliday").set(title = "Vendas com Feriados")

Também existem picos de vendas nas semanas onde não se apresentam feriados, mas sabemos que feriados importantes, como o Natal e a comemoração do dia dos Namorados, acabam afetando as vendas do mês como um todo.

In [ ]:
# Meses com feriados
print(train_data[ train_data.IsHoliday == '1']['month'].unique())

In [ ]:
# Tema do gráfico 
#sns.set_theme(style='darkgrid')
sns.set_theme(style="ticks")

# Divide a figura em 2 gráficos
plt.figure(1)

#
plt.subplot(311)
sns.boxplot( x=weekly_sales["Weekly_Sales"], y = weekly_sales["IsHoliday"], whis=[0, 100], width=.6, palette="vlag", orient = 'h')

# Add in points to show each observation
#sns.stripplot(y=weekly_sales["is_holiday"], x=weekly_sales["Weekly_Sales"], size=4, color=".3", linewidth=0)

plt.subplot(312)
sns.boxplot( x= "Weekly_Sales", y = "month", data = train_data.sort_values("month"), whis=[0, 100], width=.6, palette="vlag", orient = 'h')

# Add in points to show each observation
# sns.stripplot(y=weekly_sales["month"], x=weekly_sales["Weekly_Sales"], size=4, color=".3", linewidth=0)

plt.subplot(313)
sns.boxplot( x = "Weekly_Sales", y = "week_month", data = train_data, whis=[0, 100], width=.6, palette="vlag", orient = 'h')

# Add in points to show each observation
# sns.stripplot(y=weekly_sales["week_month"], x=weekly_sales["Weekly_Sales"], size=4, color=".3", linewidth=0)

Note que os valores das vendas tem comportamentos diferentes para cada mês. Os meses do meio do mês vem de uma crescente de vendas, com janeiro sendo o mês com menores vendas, e começa a cair tendo uma segunda crescente nos últimos dois meses do ano. Mesmo dezembro sendo o mês com feriados importantes, como o Natal, é o mês de novembro que possui o maior pico de vendas.

Em relação à variável weekly_month, não vemos comportamentos muito diferentes de vendas entre as semanas, mas temos um maior pico de vendas na quarta semana.

## Identificação da função de distribuição de probabilidades das vendas

A partir daqui, várias possibilidades foram testadas. 
Olhando para o histograma abaixo, percebemos que os dados estão muito inflacionados em valores baixos e com concentrações baixíssimas em valores muito altos. 

In [ ]:
sns.histplot( x = 'Weekly_Sales', data = train_data, bins=60)\
.set(title = 'Densidade dos Valore Reais vs Predito')

Poderíamos tentar ajustar uma distribuição gamma, que tem sua função densidade de probabilidade assimétrica, assim como a distribuição dos dados desse caso. Foi utilizada a função GLM no pacote scipy para tentar ajustar um modelo de regressão gamma, mas como é uma distribuição com três parâmetros a serem estimados, acaba sendo computacionalmente e o cluster do Kaggle não aguentou, restartando todas as vezes que tentei rodar.

Minha segunda opção foi tentar ajustar um modelo de regressão linear, mas como existem muitos valores próximos de zero nas vendas semanais, obtive muitas previsões negativas. 

Pensando que poderíamos reduzir a variabilidade dos dados, decidi aplicar a transformação de BoxCox na variável resposta e tentar ajustar um modelo de regressão linear para essa variável transformada.

### Transformação de Boxcox

A transformação de Box-Cox se dá pela seguinte expessão:

$$ y = \dfrac{x^\lambda - 1}{\lambda}, \text{para } \lambda \neq 0 $$

Nesse caso, x é o campo Weekly_Sales e y será a variável para a qual ajustaremos o modelo.

O parâmetro $\lambda$ é otimizado de forma que y se aproxime de uma distribuição normal.

O nosso valor predito voltará à escala original pela transformação inversa:

$$ x = (y\lambda + 1)^\frac{1}{\lambda} $$

In [ ]:
data["Weekly_Sales_Boxcox"] = np.nan
data["Boxcox_lambda"] = np.nan

## Aplicando tranformação de boxcox para normalização das vendas
soma_lambda = 0 # np.mean(train_data["Weekly_Sales"])
train_data["Weekly_Sales_Boxcox"], train_data["Boxcox_lambda"] = stats.boxcox( (train_data["Weekly_Sales"] + soma_lambda ) )
data["Weekly_Sales_Boxcox"][data.is_test == 0], data["Boxcox_lambda"][data.is_test == 0] = stats.boxcox( (data[data.is_test == 0] ["Weekly_Sales"] + soma_lambda ) )

# Parte 1 - Ajuste de um único modelo geral (considera as lojas e departamentos como variável aleatória)

## 1.1 - Análise de correlação/multicolinearidade para a pré-seleção de variáveis

A variável MarkDown1 ficou fortemente correlacionada com a MarkDown4 e Size; assim, ela foi descartada como input do modelo. 
Note que covariáveis com correlação alta prejudica o estimdador dos parâmetros e pode viesar o resultado final.

In [ ]:
df_corr = train_data.corr()
print( df_corr[df_corr>.5] )

## 1.2 - Ajuste do Modelo

In [ ]:
cols_dummies = ['Type', 'month', 'week_month', 'IsHoliday', 'Store', 'Dept']

x_train = pd.get_dummies( 
                train_data.drop(['Date', 'Weekly_Sales', 'Weekly_Sales_Original', 'is_test','Date2', 'Weekly_Sales_Boxcox','Weekly_Sales_Boxcox','Boxcox_lambda'], axis=1),
                columns = cols_dummies, 
                drop_first=True
            )
x_train['const'] = 1
#x_train = sm.add_constant(x_train)

y_train = train_data['Weekly_Sales_Boxcox']

x_test = pd.get_dummies( 
                test_data.drop(['Date', 'Weekly_Sales', 'Weekly_Sales_Original', 'is_test','Date2'], axis=1), 
                columns = cols_dummies, 
                drop_first=True
            )
x_test['const'] = 1
x_test = sm.add_constant(x_test)

# Ajusta as colunas do x_teste para ficar igual a x_train
x_train_names = x_train.columns
x_test_names = x_test.columns
missing_names = list(x_train_names[ ~(x_train_names.isin(x_test_names)) ])
for i in (missing_names):
    x_test[i]  = 0
x_test = x_test[x_train_names]

## 1.3 Modelo e seleção das variáveis

Uma das formas de seleção das variáveis resposta para o modelo de regressão é pelo método de stepwise. Como não encontrei um pacote em python para realizar esse método, e por questões práticas de entrega do case, parti para uma seleção manual, onde comecei retirando a variável menos significativa e ajustando um novo modelo às restantes, até que só sobrassem variáveis significativas. Dessa forma, foram excluídas do modelo final as variáveis Markdown1 e a CPI.

Pelos dados apresentarem muita variabilidade, considerei um nível de 10% de signifância para o p-valor do teste dos coeficientes, pois poderíamos acabar rejeitando alguma variável explicativa importante para o modelo caso tivesse considerado um nível de significância de 5%.

Abaixo é mostrado o summary do modelo final.

In [ ]:
features_drop = ['CPI', 'MarkDown1']
results = sm.OLS(y_train, x_train.drop( features_drop, axis = 1 ) ).fit()
results.summary()

Note que temos um valor de R2 ajustado muito bom, que nos diz o quão bom o modelo está ajustado aos dados. Quanto mais próximo de um esse coeficiente está, melhor o ajuste.

## 1.4 - Valor Ajustado

Aqui fazemos a predição para a variável transformada e invertemos a transformação para obter a variável na escala original.

In [ ]:
# Predição para a variável transformada
Weekly_Sales_Boxcox_Predict = results.predict(x_train.drop( features_drop , axis = 1) )
Boxcox_lambda = min(train_data.Boxcox_lambda)

# Predição com escala dos dados originais
Weekly_Sales_Predict = ( ((Weekly_Sales_Boxcox_Predict * Boxcox_lambda) + 1) ** (1/Boxcox_lambda) )

Weekly_Sales = train_data.Weekly_Sales
Weekly_Sales_Boxcox = train_data.Weekly_Sales_Boxcox

df_final = train_data
df_final['Weekly_Sales_Predict'] = Weekly_Sales_Predict

## 1.5 - Comparação da distribuição real versus ajustada

In [ ]:
# plt.hist( Weekly_Sales_Predict, bins=50, density = True)
# plt.hist( Weekly_Sales, bins=50, density = True )

### Modelo Ajustado

df_comparar_filtro_vendas =  \
pd.DataFrame( {
    'tipo': ['predict'] * len(Weekly_Sales_Predict),
    'y' : train_data.Weekly_Sales_Predict,
    'date': train_data.Date
}).append(
pd.DataFrame( {
    'tipo': ['y_value'] * len(y_train),
    'y' : train_data.Weekly_Sales,
    'date': train_data.Date
})  
).reset_index()

plt.figure(1)

plt.subplot(311)
# Plotando a densidade das distribuições de Weekly_Sales e de Weekly_Sales_Predict (Limitados por Weekly_Sales)
sns.kdeplot( x = 'y', hue = 'tipo', data = df_comparar_filtro_vendas)\
 .set(title = 'Densidade dos Valore Reais vs Predito', xlim=(0,40000))

plt.subplot(312)
sns.histplot( x = 'y', hue = 'tipo', data = df_comparar_filtro_vendas)\
.set(title = 'Densidade dos Valore Reais vs Predito', xlim=(0,40000))

plt.subplot(313)
sns.lineplot(  y = df_comparar_filtro_vendas.y, x = df_comparar_filtro_vendas.date, hue = df_comparar_filtro_vendas.tipo)\
.set(title = 'Valore Reais vs Predito')

Como vemos no terceiro gráfico, os valores preditos ficam um pouco abaixo dos valores reais. Isso pode estar acontecendo por causa da inflação de valores baixos. 

Na literatura existem distribuições específicas para dados inflacionados, mas não conheço nenhum pacote em python com essa solução.

## 1.6 - Gráfico da série de vendas para algumas lojas e departamentos

Abaixo plotei alguns gráficos de dos valores reais vs valores ajustados para alguns departamentos da loja 3.
Vemos que o modelo não consegue ter uma boa previsão para os picos de venda.

In [ ]:
df_comparar_lojas =  \
pd.DataFrame( {
    'Store': train_data.Store,
    'Dept': train_data.Dept,
    'tipo': ['predict'] * len(Weekly_Sales_Predict),
    'y' : train_data.Weekly_Sales_Predict,
    'date': train_data.Date
}).append(
pd.DataFrame( {
    'Store': train_data.Store,
    'Dept': train_data.Dept,
    'tipo': ['y_value'] * len(y_train),
    'y' : train_data.Weekly_Sales,
    'date': train_data.Date
})  
).reset_index()

def serie_venda_loja_departamento(loja, departamento):
    df_comparar_lojas_filtro = df_comparar_lojas[ (df_comparar_lojas.Store == loja) & (df_comparar_lojas.Dept == departamento) ]
    sns.lineplot(data = df_comparar_lojas_filtro, y="y", x = "date", hue = "tipo").set(title = "Vendas: Loja " + str(loja) + ". Departamento: " + str(departamento))

# plt.figure(2)
# for i in range(9):
#     plt.subplot(3,3,i+1)
#     serie_venda_loja_departamento(i+1, 10)
    
# plt.figure(2)
# for i in range(15):
#     plt.subplot(5,3,i+1)
#     serie_venda_loja_departamento(i+1, 2)
    
    
plt.figure(2)
for i in range(15):
    plt.subplot(5,3,i+1)
    serie_venda_loja_departamento(i+1, 3)

## 1.7 - Análise de diagnóstico (erro)

Abaixo temos alguns gráficos de análise dos resíduos do modelo. Pelo histrograma (gráfico do meio) já podemos ver que os erros não aparentam ter uma distribuição normal; além disso, vemos pelo qq-plot que os resíduos das observações extremas se afastam da distribuição teórica da normal. Tudo isso nos indica que os resíduos desse modelo não têm distribuição normal e, por isso, a regressão linear não seria o melhor modelo de ajuste aos nossos dados, mas não podemos dizer que seria um modelo totalmente descartável, pois note que, apesar de não conseguir se ajustar aos picos da série, tem um ajuste bom para o comportamento geral dela.

In [ ]:
erro_boxcox = Weekly_Sales_Boxcox_Predict - Weekly_Sales_Boxcox
erro_predict = Weekly_Sales_Predict - Weekly_Sales

plt.figure(1)

plt.subplot(131)
sns.scatterplot(Weekly_Sales_Boxcox_Predict, y_train)\
.set(title = 'Valor predito vs Valor observado')

plt.subplot(132)
sns.histplot( erro_boxcox )\
.set(title = 'Histograma dos Erros')

plt.subplot(133)
stats.probplot(erro_boxcox, dist="norm", plot=pylab)
plt.title( 'QQ-Plot dos erros (normais)' )

# plt.subplot(224)
# stats.probplot(erro_boxcox[ (erro_boxcox >= -40000) & (erro_boxcox <= 40000) ], dist="norm", plot=pylab)
# plt.title('QQ-Plot dos erros (normais) s/ outlier')

## 1.8 - Previsão

In [ ]:

# for i in x_train.columns:
#     x_test[i].astype = x_train[i].astype

y_future = results.predict( x_test.drop( features_drop , axis = 1) )
y_future = ( ((y_future * Boxcox_lambda) + 1) ** (1/Boxcox_lambda) )
data['Weekly_Sales'][data.is_test == 1] = y_future

sns.lineplot( y = data.Weekly_Sales, x = data.Date, hue = data.is_test)

# Parte 2 - Um modelo para cada loja e departamento

Como vimos que o modelo geral não consegue se ajustar muito bem à variabilidade dos dados, mesmo tendo um bom R2, parti para o ajuste de um modelo para cada loja e departamento.

A ideia de que um departamento pode ter valores de venda muito diferentes de outro pode explicar a dificuldade do modelo em prever isso, mesmo as variáveis departamento e loja sendo variáveis explicativas do modelo.

Para o ajuste desses modelos, assim como para o modelo geral, desconsiderei a variável CPI como significativa. Note que o ideal seria ter o método de seleção para cada modelo.

Para avaliarmos o ajuste dos modelos individuais, plotei, aleatoriamente, gráficos de valores reais vs valores ajustados.

Note que um modelo para cada loja e departamento se ajusta muito melhor aos dados do que um modelo geral. O modelo individual consegue prever o comportamento cíclico de alguns departamentos e até tendência de crescimento de alguns outros. A previsão futura para alguns acaba tendo picos e extrapolando os valores da série histórica, para isso poderia ser adotada uma política de identificação dessas previsões extremas e ajustar um limite igual ao máximo da série histórica ou a alguma outra melhor política para o negócio.


In [ ]:
# Variáveis para armazenar as métricas dos modelos

Boxcox_lambda = min(train_data.Boxcox_lambda)
percent_plots = 0.006

## Tabela para armazenar os parâmetros de cada modelo
df_parameters = pd.DataFrame({
    'Store' : np.nan,
    'Dept': np.nan,
    'mse_model' : np.nan,
    'mse_resid' : np.nan,
    'mse_total' : np.nan,
    'rsquared' : np.nan,
    'rsquared_adj' : np.nan
}, index = ['index'] ).dropna()


for row_store, value_store in enumerate(test_data.Store.unique()):
    for row_dept, value_dept in enumerate(test_data[test_data.Store == value_store].Dept.unique()):
        
        
        try:

            #print("Loja: " + str(value_store) + ". Dept: " +  str(value_dept))
            

            ## Define os dados para ajustar o modelo 
            cols_dummies = ['Type', 'month', 'week_month', 'IsHoliday']
            
            df_data_store_dept = data[ (data.Store ==value_store) & (data.Dept == value_dept) ].reset_index().drop(['index'], 1)
            df_data_store_dept2 = pd.get_dummies( 
                                    df_data_store_dept.drop(['Date','Store', 'Dept', 'Weekly_Sales', 'Weekly_Sales_Original','Date2', 'Weekly_Sales_Boxcox', 'Boxcox_lambda'  ], axis=1),
                                    columns = cols_dummies, 
                                    drop_first=True
                                )
            
            df_train_store_dept = df_data_store_dept2[ df_data_store_dept2.is_test == 0 ]
            df_test_store_dept = df_data_store_dept2[ df_data_store_dept2.is_test == 1 ]


            x_train = df_train_store_dept
            x_train['const'] = 1
            #x_train = sm.add_constant(x_train)

            y_train = df_data_store_dept['Weekly_Sales_Boxcox'][df_data_store_dept.is_test == 0]
            
            x_test = df_test_store_dept
            x_test['const'] = 1
            #x_test = sm.add_constant(x_test)
            
            # Ajusta as colunas do x_teste para ficar igual a x_train
            x_train_names = x_train.columns
            x_test_names = x_test.columns
            missing_names = list(x_train_names[ ~(x_train_names.isin(x_test_names)) ])
            for i in (missing_names):
                x_test[i]  = 0
            x_test = x_test[x_train_names]
            
            ## Ajuste do modelo
            query = 'results_'+ str(value_store) + '_'+ str(value_dept) + '  = sm.OLS( y_train, x_train ).fit() '
            exec(query)

            ## Salva as métricas
            df_parameters = \
            df_parameters.append(
                    pd.DataFrame({
                        'Store' : value_store,
                        'Dept': value_dept,
                        'mse_model' : eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.mse_model'),
                        'mse_resid' : eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.mse_resid'),
                        'mse_total' : eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.mse_total'),
                        'rsquared' :  eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.rsquared'),
                        'rsquared_adj' : eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.rsquared_adj')
                    },  index = ['index'] )
                )
            
            #Gráfico
            r_ajustado = eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.rsquared_adj')
            
            if ( (np.random.uniform() <= percent_plots) & (r_ajustado > 0) & ( r_ajustado < 1 ) ):
               
                exec('y_predict = results_'+ str(value_store) + '_'+ str(value_dept) + '.predict(x_train)')
                exec('y_predict_test = results_'+ str(value_store) + '_'+ str(value_dept) + '.predict(x_test)')
                
                y_predict = ( ((y_predict * Boxcox_lambda) + 1) ** (1/Boxcox_lambda) )
                y_predict_test = ( ((y_predict_test * Boxcox_lambda) + 1) ** (1/Boxcox_lambda) )
                
                y_train2 = df_data_store_dept['Weekly_Sales'][df_data_store_dept.is_test == 0]


                df_comparar_lojas = pd.DataFrame({
                        'tipo': ['predict'] * len(y_predict),
                        'y' : y_predict,
                        'date': df_data_store_dept['Date'][df_data_store_dept.is_test == 0]
                    })\
                .append(
                    pd.DataFrame({
                        'tipo': ['y_value'] * len(y_train2),
                        'y' : y_train2,
                        'date': df_data_store_dept['Date'][df_data_store_dept.is_test == 0]
                    })
                )\
                .append(
                        pd.DataFrame({
                            'tipo': ['future'] * len(y_predict_test),
                            'y' : y_predict_test,
                            'date': df_data_store_dept['Date'][df_data_store_dept.is_test == 1]
                        })
                ).reset_index().drop(['index'], 1)
                
                print(eval('results_'+ str(value_store) + '_'+ str(value_dept) + '.rsquared'))
                sns.lineplot(data = df_comparar_lojas, y="y", x = "date", hue = "tipo").set(title = "Vendas: Loja " + str(value_store) + ". Departamento: " + str(value_dept) )
                plt.show()

        except Exception as erro_except:
            print(erro_except)
                
## Score                 
q1 = np.percentile(df_parameters.rsquared_adj , 25 )
q2 = np.percentile(df_parameters.rsquared_adj , 50 )
q3 = np.percentile(df_parameters.rsquared_adj , 75 )
q4 = np.percentile(df_parameters.rsquared_adj , 90 )

df_parameters['score'] = [ 'D' if value < q1 else 'C' if value < q2 else 'B' if value < q3 else 'A' if value < q4 else 'A+' if value >= q4 else 'NA' for row, value in enumerate(df_parameters.rsquared_adj) ]


In [ ]:
df_parameters